In [74]:
# Packages
import psycopg2
from psycopg2 import OperationalError
from psycopg2 import sql

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from datetime import datetime, timedelta

import os
from dotenv import load_dotenv
dotenv_path = os.path.expanduser('~/.pwds.env')
load_dotenv(dotenv_path)



# Conexão à base de dados
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')


conn = psycopg2.connect(
                host = db_host,
                port = db_port,
                user = db_user,
                password = db_password
        )

# Flag R031 

**1. Flag Binária :** Selecionar contratos cuja diferença entre o preço base e o preço contratual seja superior a determinado treshold. 

In [75]:
def lastid():

    """
    Função que retorna o último ID copiado para a tabela auxiliar 'daily_flags'
    A obtenção deste ID tem em conta que o script correu sem problemas, os contratos foram atualizados
    na tabela 'concursos_publicos' e as flags foram todas calculadas corretamente e, consequentemente,
    todas as entradas da coluna verification são igual a true
    """

    cur = conn.cursor()
    cur.execute('''
                SELECT daily_flags."id"
                FROM daily_flags
                ORDER BY data_publicacao DESC, daily_flags."id" DESC
                LIMIT 1;
                ''')
    return(cur.fetchone()[0])

In [76]:
id = lastid()

In [82]:
def flag_r031(id_contrato, tol):

    """

    """

    cur = conn.cursor()
    cur.execute('''
                SELECT contratos_basegov."id" 
                FROM contratos_basegov
                WHERE contratos_basegov."id" > %s AND
                      ABS(anuncio_preco_base - preco_contratual) > %s
                ;
                ''', (id_contrato,tol))
    
    return(cur.fetchall())

In [84]:
flag_r031(id,100)

[(10604032,),
 (10604243,),
 (10604302,),
 (10604313,),
 (10604347,),
 (10604362,),
 (10604385,),
 (10604411,),
 (10604412,),
 (10604418,),
 (10604425,),
 (10604426,),
 (10604449,),
 (10604492,),
 (10604506,),
 (10604516,),
 (10604549,),
 (10604579,),
 (10604592,),
 (10604612,),
 (10604635,),
 (10604641,),
 (10604694,),
 (10604705,),
 (10604715,),
 (10604731,),
 (10604736,),
 (10604753,),
 (10604760,),
 (10604812,),
 (10604857,),
 (10604860,),
 (10604861,),
 (10604869,),
 (10604900,),
 (10604926,),
 (10604930,),
 (10604932,),
 (10604935,),
 (10604939,),
 (10604945,),
 (10604953,),
 (10605054,),
 (10605057,),
 (10605081,),
 (10605107,),
 (10605147,),
 (10605151,),
 (10605174,),
 (10605202,),
 (10605208,),
 (10605217,),
 (10605247,),
 (10605267,),
 (10605347,),
 (10605361,),
 (10605442,),
 (10605452,),
 (10605461,),
 (10605516,),
 (10605537,),
 (10605539,),
 (10605555,),
 (10605620,),
 (10605693,),
 (10605715,),
 (10605716,),
 (10605750,),
 (10605770,),
 (10605778,),
 (10605781,),
 (1060